In [14]:
import numpy as np
import pandas as pd
import os
import sys
import joblib
import voxelwise_tutorials
from himalaya.ridge import RidgeCV
import numpy as np
from pathlib import Path
from AOTaccess.stimulus_info_access import StimuliInfoAccess
from AOTaccess.glmsingle_access import GLMSingleAccess
from AOTanalysis.bandedRR.construct_features import (
    construct_features_motion_energy,
    construct_features_sbert_embeddings,
)
from AOTanalysis.bandedRR.construct_target import construct_target_data, construct_target_data_split_flatten_masked
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from voxelwise_tutorials.delayer import Delayer
from himalaya.backend import set_backend
from himalaya.scoring import r2_score_split

# backend = set_backend("torch_cuda", on_error="warn")
# backend = set_backend("numpy", on_error="warn")

from himalaya.kernel_ridge import MultipleKernelRidgeCV
from himalaya.kernel_ridge import Kernelizer
from sklearn import set_config
from himalaya.kernel_ridge import ColumnKernelizer
from voxelwise_tutorials.utils import explainable_variance
import nibabel as nib
from AOTglmsingle.glmoutput_save_nifti import get_affine_matrix
from AOTaccess.fmriprep_access import FmriprepAccess
from nilearn.plotting import view_img

import joblib


test_data_index = 2


sub = 1

video_betas, video_index = construct_target_data_split_flatten_masked(
    sub, split_num=10, index=2
)
motion_energy_features = construct_features_motion_energy(video_index)
sbert_embeddings = construct_features_sbert_embeddings(video_index)


# concatenate the features
Xtest_motion_energy = motion_energy_features
Xtest_sbert = sbert_embeddings
print(f"Shape of X: {Xtest_motion_energy.shape}")
print(f"Shape of X: {Xtest_sbert.shape}")
ytest = video_betas
print(f"Shape of y: {ytest.shape}")


def clean_data(data):
    data = np.nan_to_num(data, nan=0.0, posinf=0, neginf=0)
    return data.astype(np.float32)


Xtest_motion_energy = clean_data(Xtest_motion_energy)
Xtest_sbert = clean_data(Xtest_sbert)
ytest = clean_data(ytest)

Loaded beta from /tank/shared/2024/visual/AOT/derivatives/glmsingle/video_betas/sub-001/0001_fw_betas.nii
Shape of beta: (2, 86, 104, 88)
Loaded betas for video 1
Shape of betas: (2, 86, 104, 88)
Loaded beta from /tank/shared/2024/visual/AOT/derivatives/glmsingle/video_betas/sub-001/0002_fw_betas.nii
Shape of beta: (2, 86, 104, 88)
Loaded betas for video 2
Shape of betas: (2, 86, 104, 88)
Loaded beta from /tank/shared/2024/visual/AOT/derivatives/glmsingle/video_betas/sub-001/0003_fw_betas.nii
Shape of beta: (2, 86, 104, 88)
Loaded betas for video 3
Shape of betas: (2, 86, 104, 88)
Loaded beta from /tank/shared/2024/visual/AOT/derivatives/glmsingle/video_betas/sub-001/0004_fw_betas.nii
Shape of beta: (2, 86, 104, 88)
Loaded betas for video 4
Shape of betas: (2, 86, 104, 88)
Loaded beta from /tank/shared/2024/visual/AOT/derivatives/glmsingle/video_betas/sub-001/0005_fw_betas.nii
Shape of beta: (2, 86, 104, 88)
Loaded betas for video 5
Shape of betas: (2, 86, 104, 88)
Loaded beta from /ta

In [15]:

affine = get_affine_matrix(sub=1, ses=1)

affine matrix source path: /tank/shared/2024/visual/AOT/derivatives/fmripreps/aotfull_preprocs/fullpreproc_forcesyn_endfix/sub-001/ses-01/func/sub-001_ses-01_task-AOT_rec-nordicstc_run-1_space-T1w_part-mag_boldref.nii.gz


In [16]:
model_motion_path = (
    "/tank/shared/2024/visual/AOT/temp/bandedRR_split/model_sub1_split0_motion.joblib"
)
model_semantic_path = (
    "/tank/shared/2024/visual/AOT/temp/bandedRR_split/model_sub1_split0_semantic.joblib"
)


voxel_num = 787072

model_motion = joblib.load(model_motion_path)
model_semantic = joblib.load(model_semantic_path)

score_motion = model_motion.score(Xtest_motion_energy, ytest)
score_semantic = model_semantic.score(Xtest_sbert, ytest)

print("shape of score_motion:", score_motion.shape)
print("shape of score_semantic:", score_semantic.shape)

def reshape_from_flatten_masked_to_wholebrain(data, sub):
    glmaccess = GLMSingleAccess()
    mask = glmaccess.read_R2_mask(sub, ses=1)
    flatten_mask = mask.flatten()
    data_reshape = np.zeros(voxel_num)
    data_reshape[flatten_mask] = data
    data_reshape = data_reshape.reshape(86, 104, 88)
    return data_reshape

# glmaccess = GLMSingleAccess()
# mask = glmaccess.read_R2_mask(sub, ses=1)
# flatten_mask = mask.flatten()

# motion_score_reshape = np.zeros(voxel_num)
# semantic_score_reshape = np.zeros(voxel_num)

# motion_score_reshape[flatten_mask] = score_motion
# semantic_score_reshape[flatten_mask] = score_semantic

# motion_score_reshape = motion_score_reshape.reshape(86, 104, 88)
# semantic_score_reshape = semantic_score_reshape.reshape(86, 104, 88)

motion_score_reshape = reshape_from_flatten_masked_to_wholebrain(score_motion, sub)
semantic_score_reshape = reshape_from_flatten_masked_to_wholebrain(score_semantic, sub)

print("shape of motion_score_reshape:", motion_score_reshape.shape)
print("shape of semantic_score_reshape:", semantic_score_reshape.shape)


savedir = "/tank/shared/2024/visual/AOT/temp/bandedRR_splitmeanscore"

motionscoreimg = nib.Nifti1Image(motion_score_reshape, affine)
nib.save(motionscoreimg, os.path.join(savedir, "motion_score.nii.gz"))

semanticscoreimg = nib.Nifti1Image(semantic_score_reshape, affine)
nib.save(semanticscoreimg, os.path.join(savedir, "semantic_score.nii.gz"))

shape of score_motion: (641846,)
shape of score_semantic: (641846,)
Loaded R2 from /tank/shared/2024/visual/AOT/derivatives/glmsingle/mainexp/sub-001_ses-01_T1W_nordicstc/TYPED_FITHRF_GLMDENOISE_RR/R2.nii
Shape of R2: (86, 104, 88)
Shape of R2 mask: (86, 104, 88)
shape of motion_score_reshape: (86, 104, 88)
shape of semantic_score_reshape: (86, 104, 88)
